In [1]:
import os

In [2]:
os.chdir('..')

<img src="flow_1.png">

In [3]:
from flows.flows import Flows

/anaconda3/envs/ml-nav/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


Welcome to the Data Science Package. First create an object as follows:
For example, use the code below to import the flow 0:
flow = Flows(0)
You can define the `categorical_threshold` which is the maximum number of categories that a categorical feature should have before considering it as continuous numeric feature. The default value is 50
For example, use the code below to import the flow 0 with defining the categorical_threshold as 50
flow = Flows(flow_id=0, categorical_threshold=50)


In [4]:
flow = Flows(1)

Please use the following function to read the data
dataframe_dict = flow.load_data(path: str, files_list: list)
For example: path = './data'
For example: files_list = ['train.csv','test.csv']
The output is a dictionary that contains dataframes e.g.  
dataframe_dict = {'train': train_dataframe,'test': test_dataframe}


In [5]:
path = './data/flow_1'
files_list = ['train.csv','test.csv']

In [6]:
dataframe_dict, columns_set = flow.load_data(path, files_list)

A summary of the data sets


,train,test
column type,,
categorical_integer,18,18
categorical_string,43,43
continuous,20,19
date,0,0
json,0,0
other,0,0
total amount,81,80


NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
The possible ids are {'Id'}
The possible possible_target are ['SalePrice']
The type of the problem that should be solved {'SalePrice': 'regression'}
If you have categorical features with string labels, Encode the categorical features by applying the following function:
dataframe_dict, columns_set = flow.encode_categorical_feature(dataframe_dict: dict)


In [7]:
dataframe_dict, columns_set = flow.encode_categorical_feature(dataframe_dict)

The reference dataframe is: train
******************************
A summary of the data sets


,train,test
column type,,
categorical_integer,61,61
categorical_string,0,0
continuous,20,19
date,0,0
json,0,0
other,0,0
total amount,81,80


NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
You have categorical features. Apply one-hot encoding to the categorical features by applying the following function:
dataframe_dict, columns_set = flow.one_hot_encoding (dataframe_dict: dict, ignore_columns: list, class_number_range=[3, 50]
Since one-hot encoding can produce a lot of features, class_number_range will limit the encoding process only for features which have between 3 and 49 unique values.
If you are solving a classification problem, you should exclude the target from the one-hot encoding process by defining the ingore_columns
ingore_columns = [<your target/label>]
You can add more columns to the ingore_columns list to ignore


In [8]:
ignore_columns = ['id', 'SalePrice']

In [9]:
dataframe_dict, columns_set = flow.one_hot_encoding(dataframe_dict,
                                                    "train",
                                                    ignore_columns,
                                                    class_number_range=[3, 50])

A summary of the data sets


,train,test
column type,,
categorical_integer,436,436
categorical_string,0,0
continuous,20,19
date,0,0
json,0,0
other,0,0
total amount,456,455


NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
If you have numeric features, it is a good idea to normalize numeric features. Use the following function for feature normalization :
dataframe_dict, columns_set = flow.scale_data (dataframe_dict: dict, ignore_columns: list)
For example: ignore_columns = ['id', 'target']


In [10]:
dataframe_dict, columns_set = flow.scale_data(dataframe_dict, ignore_columns)

A summary of the data sets


,train,test
column type,,
categorical_integer,436,436
categorical_string,0,0
continuous,20,19
date,0,0
json,0,0
other,0,0
total amount,456,455


NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
Your features are ready to train the model: 
If you want to explore the data you can run one of the following functions: 
1 . flow.exploring_data(dataframe_dict: dict, key_i: str)
For example: flow.exploring_data(dataframe_dict, 'train')
2 . flow.comparing_statistics(dataframe_dict: dict)
For example: flow.comparing_statistics(dataframe_dict)




You can start training the model by applying the following function: 
model_index_list, save_models_dir, y_test = flow.training(parameters)
parameters = { 
 "data": {
 "train": {"features": train_dataframe, "target": train_target},
 "valid": {"features": valid_dataframe, "target": valid_target},
 "test": {"features": test_dataframe, "target": test_target},
 },
 "split": {
 "method": "split",  # "

In [11]:
ignore_columns = ["id", "SalePrice"]
columns = dataframe_dict["train"].columns
train_dataframe = dataframe_dict["train"][[x for x in columns_set["train"]["categorical_integer"] if x not in ignore_columns]]
test_dataframe = dataframe_dict["test"][[x for x in columns_set["train"]["categorical_integer"] if x not in ignore_columns]]
train_target = dataframe_dict["train"]["SalePrice"]

In [12]:
parameters = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "split",  # "method":"kfold"
        "split_ratios": 0.2,  # foldnr:5 , "split_ratios": 0.2 # "split_ratios":(0.3,0.2)
    },
    "model": {"type": "Ridge linear regression",
              "hyperparameters": {"alpha": "optimize",  # alpha:optimize
                                  },
              },
    "metrics": ["r2_score", "mean_squared_error"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

In [13]:
model_index_list, save_models_dir, y_test = flow.training(parameters)

the optimized alpha value 1.0


,model 0
mean_squared_error (train.train),4.511490e+08
mean_squared_error (train.validation_0),1.099440e+09
r2_score (train.train),9.244000e+01
r2_score (train.validation_0),8.567000e+01


,model 0
mean_squared_error (train),5.808072e+08
r2_score (train),9.079000e+01


This is the end of the flow


In [14]:
parameters_lighgbm = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "kfold",  # "method":"kfold"
        "fold_nr": 5,  # foldnr:5 , "split_ratios": 0.2 # "split_ratios":(0.3,0.2)
    },
    "model": {"type": "lightgbm",
              "hyperparameters": dict(objective='regression', metric='root_mean_squared_error', num_leaves=5,
                                      boost_from_average=True,
                                      learning_rate=0.05, bagging_fraction=0.99, feature_fraction=0.99, max_depth=-1,
                                      num_rounds=10000, min_data_in_leaf=10, boosting='dart')
              },
    "metrics": ["r2_score", "mean_squared_error"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

In [15]:
model_index_list, save_models_dir, y_test = flow.training(parameters_lighgbm)

shuffle is not provided: 'shuffle'
random_state is not provided: 'random_state'
fold_nr. 1
{'r2_score (train.train)': 98.6, 'mean_squared_error (train.train)': 85407084.72653972, 'r2_score (train.validation)': 89.69, 'mean_squared_error (train.validation)': 735456111.1847914}
fold_nr. 2
{'r2_score (train.train)': 98.82, 'mean_squared_error (train.train)': 75297508.23059897, 'r2_score (train.validation)': 85.56, 'mean_squared_error (train.validation)': 876638566.3950267}
fold_nr. 3
{'r2_score (train.train)': 98.81, 'mean_squared_error (train.train)': 76437480.58935387, 'r2_score (train.validation)': 81.49, 'mean_squared_error (train.validation)': 1082012028.1759164}
fold_nr. 4
{'r2_score (train.train)': 98.7, 'mean_squared_error (train.train)': 80459577.24151137, 'r2_score (train.validation)': 81.24, 'mean_squared_error (train.validation)': 1280155802.525103}
fold_nr. 5
{'r2_score (train.train)': 98.78, 'mean_squared_error (train.train)': 79321514.48982576, 'r2_score (train.validation)'

,fold_1,fold_2,fold_3,fold_4,fold_5,mean
mean_squared_error (train.train),8.540708e+07,7.529751e+07,7.643748e+07,8.045958e+07,7.932151e+07,7.938463e+07
mean_squared_error (train.validation),7.354561e+08,8.766386e+08,1.082012e+09,1.280156e+09,6.733924e+08,9.295310e+08
r2_score (train.train),9.860000e+01,9.882000e+01,9.881000e+01,9.870000e+01,9.878000e+01,9.874200e+01
r2_score (train.validation),8.969000e+01,8.556000e+01,8.149000e+01,8.124000e+01,8.802000e+01,8.520000e+01


,model 1,model 2,model 3,model 4,model 5,mean
mean_squared_error (train),2.154169e+08,2.355657e+08,2.775524e+08,3.203988e+08,1.981357e+08,2.494139e+08
r2_score (train),9.658000e+01,9.626000e+01,9.560000e+01,9.492000e+01,9.686000e+01,9.604400e+01


This is the end of the flow


In [16]:
parameters_xgboost = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "kfold",  # "method":"kfold"
        "fold_nr": 5,  # fold_nr:5 , "split_ratios": 0.3 # "split_ratios":(0.3,0.2)
    },
    "model": {"type": "xgboost",
              "hyperparameters": {'max_depth': 5, 'eta': 1, 'eval_metric': "rmse"}
              },
    "metrics": ["r2_score", "mean_squared_error"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

In [17]:
model_index_list, save_models_dir, y_test = flow.training(parameters_xgboost)

shuffle is not provided: 'shuffle'
random_state is not provided: 'random_state'
The objective is not defined. The default value is reg:squarederror. Error: 'objective'
fold_nr. 1
The num_round is not defined. The default value is num_round = 10. Error: 'num_round'
[0]	train-rmse:40632.8	test-rmse:51489.7
[1]	train-rmse:31553	test-rmse:46632.6
[2]	train-rmse:28335.7	test-rmse:44061.5
[3]	train-rmse:25816.4	test-rmse:45387.2
[4]	train-rmse:23930.6	test-rmse:45457.9
[5]	train-rmse:22649.7	test-rmse:44379.5
[6]	train-rmse:21681.1	test-rmse:44091.1
[7]	train-rmse:20877.6	test-rmse:44202.4
[8]	train-rmse:19857.5	test-rmse:44549.4
[9]	train-rmse:18202.9	test-rmse:44142.4
{'r2_score (train.train)': 94.56, 'mean_squared_error (train.train)': 331344493.8129808, 'r2_score (train.validation)': 72.68, 'mean_squared_error (train.validation)': 1948553713.7916}
fold_nr. 2
The num_round is not defined. The default value is num_round = 10. Error: 'num_round'
[0]	train-rmse:43017.9	test-rmse:43932.9
[1]	

,fold_1,fold_2,fold_3,fold_4,fold_5,mean
mean_squared_error (train.train),3.313445e+08,3.017587e+08,2.853729e+08,3.355912e+08,3.643770e+08,3.236889e+08
mean_squared_error (train.validation),1.948554e+09,1.841913e+09,1.789568e+09,1.660958e+09,1.451202e+09,1.738439e+09
r2_score (train.train),9.456000e+01,9.526000e+01,9.555000e+01,9.457000e+01,9.438000e+01,9.486400e+01
r2_score (train.validation),7.268000e+01,6.965000e+01,6.938000e+01,7.566000e+01,7.418000e+01,7.231000e+01


,model 1,model 2,model 3,model 4,model 5,mean
mean_squared_error (train),6.547863e+08,6.097896e+08,5.862120e+08,6.006645e+08,5.817420e+08,6.066389e+08
r2_score (train),8.962000e+01,9.033000e+01,9.071000e+01,9.048000e+01,9.078000e+01,9.038400e+01


This is the end of the flow
